In [1]:
import requests
import pandas as pd
import json
from cred import credentials
from request_api import get_genesis_api
from sql import get_data

In [2]:
data = pd.read_csv('data/all_merged_scaled.csv', index_col=0)
data

,year,state,age_avg,total_population,country,new_building,area_Wg,rooms_Wg,kino,kino_seat,...,scaled_area_Wg,scaled_rooms_Wg,scaled_kino,scaled_kino_seat,scaled_museum,scaled_apartment,scaled_restaurant,score,age_avg_per_resident,scaled_age_avg
0,2020,Baden-Württemberg,43.240650,11103043,Germany,26600,1402,186843,252.0,100970.0,...,0.852220,0.618560,0.220814,0.142081,0.784282,0.000000,0.556773,3.844323,0.000004,0.023827
1,2020,Bayern,43.488237,13140183,Germany,40427,1948,283342,284.0,125431.0,...,1.000000,1.000000,0.188775,0.189097,0.337564,0.132047,0.710754,4.499136,0.000003,0.014237
2,2020,Berlin,42.149704,3664088,Germany,3043,156,53307,95.0,48373.0,...,0.289389,0.434919,0.316358,0.569664,0.268316,0.520919,1.000000,3.548144,0.000012,0.148578
3,2020,Brandenburg,46.722180,2531071,Germany,8237,131,53611,64.0,24371.0,...,0.351131,0.969237,0.297385,0.197749,0.974735,0.445386,0.100073,4.598319,0.000018,0.262623
4,2020,Bremen,43.143783,680130,Germany,778,35,6223,12.0,11791.0,...,0.349140,0.000000,0.071390,1.000000,0.262221,0.466226,0.643403,3.921681,0.000063,1.000000
5,2020,Hamburg,41.608811,1852478,Germany,2221,139,31096,33.0,20595.0,...,0.507667,0.615183,0.076425,0.352711,0.000000,0.394256,0.988801,3.415281,0.000022,0.328232
6,2020,Hessen,43.518179,6293154,Germany,12237,494,97673,128.0,56589.0,...,0.530957,0.513223,0.151114,0.131490,0.386893,0.051506,0.645965,2.944097,0.000007,0.073352
7,2020,Mecklenburg-Vorpommern,46.846078,1610774,Germany,3454,68,24681,79.0,19778.0,...,0.286971,0.497267,1.000000,0.473557,1.000000,0.804288,0.707851,5.748766,0.000029,0.436798
8,2020,Niedersachsen,44.270348,8003421,Germany,22024,714,155134,177.0,69454.0,...,0.603004,0.824418,0.203634,0.098796,0.577593,0.209557,0.287010,3.647350,0.000006,0.050665
9,2020,Nordrhein-Westfalen,43.759486,17925570,Germany,30352,821,228644,273.0,171030.0,...,0.311077,0.290452,0.000000,0.188630,0.137301,0.219996,0.308391,1.811780,0.000002,0.000000


In [12]:
columns = ['new_building', 'area_Wg', 'rooms_Wg', 'kino', 'kino_seat', 'museum',
       'apartment', 'restaurant']

default_weights = dict(zip(columns, [1] * 8))

def calc_score(weights=default_weights):
    data['w_score'] = [0] * 16
    for col in columns:
        data['w_score'] += data['scaled_%s' % col] * weights[col]
    return data[['state', 'w_score']].sort_values(by='w_score', ascending=False)

In [14]:
calc_score()

,state,w_score
7,Mecklenburg-Vorpommern,5.311969
1,Bayern,4.484899
3,Brandenburg,4.335696
14,Schleswig-Holstein,4.134553
10,Rheinland-Pfalz,3.861403
0,Baden-Württemberg,3.820496
8,Niedersachsen,3.596684
12,Sachsen,3.469268
2,Berlin,3.399566
5,Hamburg,3.087049


In [13]:
calc_score({'new_building':2, 'area_Wg':2, 'rooms_Wg':2, 'kino':1, 'kino_seat':1, 'museum':1,
       'apartment':1, 'restaurant':1})

,state,w_score
1,Bayern,7.411561
3,Brandenburg,6.656064
7,Mecklenburg-Vorpommern,6.638242
14,Schleswig-Holstein,6.610626
0,Baden-Württemberg,5.937041
10,Rheinland-Pfalz,5.861726
8,Niedersachsen,5.816780
12,Sachsen,4.617819
6,Hessen,4.374524
5,Hamburg,4.361904
